# ライブラリのインポート

In [12]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread
from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent

from azure.ai.agents.models import FileInfo, FileSearchTool, VectorStore

# 環境変数の取得

In [15]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID=os.getenv("FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID")
FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")
FOUNDRY_MCP_AGENT_ID=os.getenv("FOUNDRY_MCP_AGENT_ID")
FOUNDRY_FILE_SEARCH_AGENT_ID=os.getenv("FOUNDRY_FILE_SEARCH_AGENT_ID")

# ユーティリティ関数

In [5]:
import datetime
import json
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.contents.text_content import TextContent


async def print_thread_message_details(thread: str):
    """
    スレッドのメッセージ詳細を表示します。

    Args:
        thread (str): スレッドのインスタンス
    """
    async for message in thread.get_messages():
        print("-----")

        for item in message.items:
            if isinstance(item, FunctionCallContent):
                print(f"[Function Calling] by {message.ai_model_id}")
                print(f" - Function Name : {item.name}")
                print(f" - Arguments     : {item.arguments}")

            elif isinstance(item, FunctionResultContent):
                print(f"[Function Result]")
                # 文字列のデコード変換
                if isinstance(item.result, str):
                    try:
                        decoded = json.loads(item.result)
                        print(f" - Result        : {decoded}") # デコード成功時は変換後の値を表示
                    except json.JSONDecodeError:
                        print(f" - Result        : {item.result}")  # デコード失敗時はそのまま
                else:
                    print(f" - Result        : {item.result}")

            elif isinstance(item, TextContent):
                if message.name:
                    print(f"[Agent Response] from {message.ai_model_id}")
                else:
                    print("[User Message]")
                print(f" - Content       : {item.text}")

            else:
                print(f"[Unknown Item Type] ({type(item).__name__})")
                print(f" - Raw Item      : {item}")


def log_with_timestamp(message: str) -> None:
    """
    現在時刻付きでメッセージを標準出力にログとして表示します。

    Args:
        message (str): 出力するログメッセージ。
    """
    timestamp = datetime.datetime.now().strftime("%H:%M:%S.%f")[:-3]
    print(f"[{timestamp}] {message}")


# プロジェクトクライアントの初期化

In [6]:
# AzureAIAgent クライアントを初期化
project_client = AzureAIAgent.create_client(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# Foundry 上のエージェントを取得する方法
※ 先ほど作成したエージェントを取得します。

In [16]:
file_search_agent_definition = await project_client.agents.get_agent(agent_id=FOUNDRY_FILE_SEARCH_AGENT_ID)

In [17]:
agent = AzureAIAgent(
    client=project_client,
    definition=file_search_agent_definition,
)

In [18]:
thread = AzureAIAgentThread(
    client=project_client
)
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: None


In [19]:
message= (
    "パソコンの保障プランでもっとも安いプランは？"
)

response = await agent.get_response(
    messages=[message],
    thread=thread,
)

print(response)

パソコンの保障プランでもっとも安いプランは「1年プラン」で、料金は税込10,000円です。内容としては、修理費用全額カバー、代替機の提供（最短3営業日）、データ復旧（論理障害のみ）、24時間365日サポートなどが含まれています【4:3†パソコン保障プラン.html】。


# エージェントの作成

# Foundry 上のエージェントを取得する方法
※ 先ほど作成したエージェントを取得します。

In [ ]:
code_interpreter_agent_definition = await project_client.agents.get_agent(agent_id=FOUNDRY_CODE_INTERPRETER_AGENT_ID)


In [11]:
code_interpreter_agent_definition

{'id': 'asst_BCUSg2MD4dWVlfuATOeUG6gx', 'object': 'assistant', 'created_at': 1753669174, 'name': 'code_interpreter_agent', 'description': 'Code Interpreter を利用して、計算や図表の出力に特化した分析アシスタントです。', 'model': 'gpt-4.1', 'instructions': 'あなたは日本国内の気温や地理情報に答えたり、計算や図表の出力に特化したアシスタントです。利用可能なツールを確認し、必要に応じて使用してください。なお、グラフを描画する際は日本語が文字化けするため、ラベルやタイトルは英語で出力してください。', 'tools': [{'type': 'code_interpreter'}, {'type': 'function', 'function': {'name': 'get_current_time_jst', 'description': '日本標準時（JST, UTC+9）の今日の日付を "YYYY-MM-DD" 形式の文字列で返します。日時情報は変更しません。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_prefecture_location', 'description': '都道府県名から緯度・経度を取得します。都道府県一覧・座標は変更しません。', 'parameters': {'type': 'object', 'properties': {'prefecture': {'type': 'string', 'description': '都道府県名（例: "東京都"）'}}, 'required': ['prefecture']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_temperature', 'description': '指定した緯度・経度と日付

# AzureAIAgent クライアントの初期化

In [8]:
agent = AzureAIAgent(
    client=project_client,
    definition=code_interpreter_agent_definition,
)

# スレッドの作成

In [9]:
thread = AzureAIAgentThread(
    client=project_client
)
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: None


## レスポンスを取得

In [10]:
message= (
    "初期投資額が100万円、年利回りが3.5%、毎年10万円ずつ追加投資を"
    "20年間継続したときの最終的な資産額を計算し、円単位で四捨五入して答えてください。"
)

response = await agent.get_response(
    messages=[message],
    thread=thread,
)

print(response)

AgentInvokeException: The following function tool(s) are defined on the agent but missing from the kernel: ['get_current_time_jst', 'get_prefecture_location', 'get_temperature']. Please ensure all required tools are registered with the kernel.

In [101]:
message = "グラフで出力して下さい。また、単純に投資せずに貯蓄した際と比較してください。"

response = await agent.get_response(
    messages=[message],
    thread=thread,
)

print(response)

上記のグラフは、20年間にわたり毎年同額を貯めた場合と、年利3.5%で投資した場合の資産額の推移を比較したものです。  
- 橙色の線「With Investment」は投資の場合、  
- 赤色の線「Without Investment」は単純な貯蓄の場合を示しています。

投資をすることで資産が大きく増加することが分かります。


In [95]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
)
from IPython.display import Image, display

In [20]:
from azure.ai.agents.models import ListSortOrder
import os
from IPython.display import Image, display

async def agent_run_outputs(thread_id, agents_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    async for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")

        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                await agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))


In [21]:
await agent_run_outputs(thread.id, project_client.agents)

USER: パソコンの保障プランでもっとも安いプランは？
ASSISTANT: パソコンの保障プランでもっとも安いプランは「1年プラン」で、料金は税込10,000円です。内容としては、修理費用全額カバー、代替機の提供（最短3営業日）、データ復旧（論理障害のみ）、24時間365日サポートなどが含まれています【4:3†パソコン保障プラン.html】。


In [22]:
await print_thread_message_details(thread)

-----
[Agent Response] from None
 - Content       : パソコンの保障プランでもっとも安いプランは「1年プラン」で、料金は税込10,000円です。内容としては、修理費用全額カバー、代替機の提供（最短3営業日）、データ復旧（論理障害のみ）、24時間365日サポートなどが含まれています【4:3†パソコン保障プラン.html】。
[Unknown Item Type] (AnnotationContent)
 - Raw Item      : AnnotationContent(type=file_citation, file_id=assistant-JK6ipBT5kMo8fRCYdw7adD, url=None, quote=【4:3†パソコン保障プラン.html】, start_index=113, end_index=133)
-----
[Agent Response] from None
 - Content       : パソコンの保障プランでもっとも安いプランは？
